In [1]:
import sys, os, json
import pandas as pd
from sqlalchemy import create_engine
import sys;
import xml.dom.minidom;
import json
import jmespath
import time
import duckdb
import numpy as np
import re

sys.path.insert(1, "../../src")
import exp_trade_attr_builder as exp_trade

%load_ext autoreload
%autoreload 2

#### create flag based on raw table

In [2]:
sql_base = """
with a as (select *,
                  try_cast(TermsDuration as int)                                                      as terms,
                  case
                      when ((EnhancedAccountTypeCode in
                             ('00', '01', '02', '03', '06', '09', '0A', '0F', '10', '11', '12', '13', '14', '17', '1A',
                              '1B', '1C', '20', '21', '22', '23', '30', '31', '3A', '65', '66', '67', '68', '69', '6A',
                              '78', '7B'))
                          and not
                                ((EnhancedAccountTypeCode in ('3A')
                                    or
                                  (EnhancedAccountTypeCode in ('13')
                                      and
                                   KOBCode in ('AC', 'AF', 'AL', 'AN', 'AU', 'AZ', 'BB', 'BS', 'FA', 'FC', 'FF', 'FP', 'FS',
                                           'FZ')))
                                    or
                                 (EnhancedAccountTypeCode in ('00')
                                     or
                                  (KOBCode in ('AF', 'AL', 'AN', 'AU', 'AZ', 'FA')
                                      and
                                   EnhancedAccountTypeCode in
                                   ('00', '01', '02', '03', '06', '09', '0A', '0F', '10', '11', '12', '14', '17', '1A',
                                    '1B', '1C', '20', '21', '22', '23', '30', '31', '65', '66', '67', '68', '69', '6A',
                                    '78', '7B'))))
                          and not
                                (substr(KOBCode, 1, 1) in ('E')
                                    or
                                 EnhancedAccountTypeCode in ('12'))) then 1
                      else 0 end                                                              as flag_psl,
                  case
                      when ((KOBCode in ('BB', 'BC', 'BN', 'BO', 'BS', 'FC', 'FS', 'NF', 'NS', 'NU', 'NZ')
                          and
                             TermsDuration in ('LOC', 'REV', 'UNK')
                          and
                             EnhancedAccountTypeCode in ('18', '37'))
                          or
                            (KOBCode in ('BB', 'BC', 'BN', 'BO', 'BS', 'FC', 'FS', 'NF', 'NS', 'NU', 'NZ')
                                and
                             EnhancedAccountTypeCode in ('0G'))
                          or
                            (KOBCode in ('BB', 'BC', 'FC', 'FS', 'NF', 'NS', 'NU')
                                and
                             TermsDuration in ('LOC', 'REV')
                                and
                             EnhancedAccountTypeCode in ('15', '47')
                                and
                             (case
                                  when try_cast(AmountValue_1 as int) > 0 then try_cast(AmountValue_1 as int)
                                  else try_cast(AmountValue_2 as int) end <= 25000))) then 1
                      else 0 end                                                              as flag_bca,
                  datepart('month',age(cast(credit_pull_date as date), cast(OpenDate as date)))+ datepart('year',age(cast(credit_pull_date as date), cast(OpenDate as date)))*12 as seasoning_month,
                  coalesce(BalanceAmount, AmountPastDue) as effective_bal,
                  case
                      when substring(PaymentProfile, 1, 1) similar to ('1|2|3|4|5|6') then 1
                      else 0 end                                                              as flag_currentdq,
                  case
                      when DelinquenciesOver90Days>0 or DelinquenciesOver60Days>0 or DelinquenciesOver30Days>0 or DerogCounter>0 then 1
                      else 0
                      end                                                                     as flag_everdq,
                  try_cast(MonthlyPaymentAmount as int)                                     as payment_schedule
           from trade_line_df
)
"""

In [3]:
param_config = {'descriptor': ['num_trade', 'num_month_recent', 'total_bal', 'ever_dq', 'current_dq', 'avg_term'],
  'source_table': ['tradeline'],
  'loan_type': ['flag_psl'],
  'date_window':['','1month','3month','6month','12month'],
  'sublist': ['',"('PROSPER MARKETPLACE IN','LENDING CLUB CORP','ONEMAIN','ONEMAIN FINANCIAL','UPSTART NETWORK INC.','DISCOVER PERSONAL LOAN','BEST EGG/SST','BEST EGG','POSSIBLE FINANCIAL INC', 'FREEDOM PLUS','OPPLOANS/FINWISE','OPP LOANS')"],
  'openorclosed': ['','Open', 'Closed']
}

In [4]:
#  generate list of parameters based on parameter config file
param_list = []
trade_dict = pd.DataFrame()
for i in range(0,len(param_config['descriptor'])):
        for j in range(0,len(param_config['source_table'])):
                for k in range(0, len(param_config['loan_type'])):
                        for l in range(0, len(param_config['date_window'])):
                                for m in range(0, len(param_config['sublist'])):
                                    for n in range(0, len(param_config['openorclosed'])):
                                        # define descriptor
                                        descriptor = param_config['descriptor'][i]
                                        # define source_table
                                        source_table = param_config['source_table'][j]
                                        # define loan_type
                                        loan_type = param_config['loan_type'][k]
                                        # define window
                                        try:
                                            date_window = param_config['date_window'][l]
                                        except:
                                            date_window = ''
                                        # define sublist
                                        sublist = param_config['sublist'][m]
                                        # define openorclosed
                                        openorclosed = param_config['openorclosed'][n]
                                        # define parameters
                                        param_new = {
                                                    'descriptor': descriptor,
                                                    'source_table': source_table,
                                                    'loan_type': loan_type,
                                                    'date_window': date_window,
                                                    'sublist': sublist,
                                                    'openorclosed': openorclosed}
                                        attr_0 = exp_trade.exp_trade_attr_builder(param_new)
                                        # field_name 
                                        field_name = attr_0.get_attr_name()  
                                        # long description
                                        long_description = attr_0.get_attr_name(1)
                                        # generate list of parameters based on parameter config file
                                        param_new.update({'field_name': field_name,
                                                    'long_description': long_description})
                                        trade_dict_new = pd.DataFrame({'field_name':[field_name],'long_description':[long_description]})
                                        if param_new not in param_list:
                                            param_list.append(param_new)
                                            trade_dict = trade_dict.append(trade_dict_new)

In [5]:
len(param_list)

180

In [6]:
# generate sql code for aggregation section
sql_agg = ""
for params_trade in param_list:
    attr = exp_trade.exp_trade_attr_builder(params_trade)
    sql_agg = sql_agg +","  + attr.get_trade_stats_sql() 
    if params_trade['field_name'] == 'sofi_num_month_recent_list_psl':
        print(attr.get_trade_stats_sql())

case when sum(flag_psl*1*(case when SubscriberDisplayName in ('PROSPER MARKETPLACE IN','LENDING CLUB CORP','ONEMAIN','ONEMAIN FINANCIAL','UPSTART NETWORK INC.','DISCOVER PERSONAL LOAN','BEST EGG/SST','BEST EGG','POSSIBLE FINANCIAL INC', 'FREEDOM PLUS','OPPLOANS/FINWISE','OPP LOANS') then 1 else 0 end)*1)>0 then min(case when flag_psl*1*(case when SubscriberDisplayName in ('PROSPER MARKETPLACE IN','LENDING CLUB CORP','ONEMAIN','ONEMAIN FINANCIAL','UPSTART NETWORK INC.','DISCOVER PERSONAL LOAN','BEST EGG/SST','BEST EGG','POSSIBLE FINANCIAL INC', 'FREEDOM PLUS','OPPLOANS/FINWISE','OPP LOANS') then 1 else 0 end)*1=1 then seasoning_month else null end) else null end as sofi_num_month_recent_list_psl


In [7]:
sql = sql_base + """select id, applicant_type, credit_pull_id
--<placeholder>--
from a 
group by 1, 2, 3
"""

sql = sql.replace("--<placeholder>--", sql_agg)

##### loop through each| quarter and generate inquiry attributes

In [8]:
quarter_list = ['2021Q2', '2021Q3', '2021Q4', '2022Q1']

In [9]:
trade_attr = pd.DataFrame()

for quarter in quarter_list:
    print(quarter)
    trade_line_df = pd.read_parquet(f's3://sofi-data-science/hpeng/pl-gen4/data_dump/raw_tables/feature_creation/tradeline_{quarter}.parquet')
    trade_attr_new = duckdb.query(sql).to_df()
    trade_attr = trade_attr.append(trade_attr_new)
    
trade_attr = trade_attr.drop_duplicates()
trade_attr.to_parquet(f's3://sofi-data-science/hpeng/pl-gen4/data_dump/raw_tables/feature_creation/agg_trade_attr_2021Q1_202201.parquet')

2021Q2
2021Q3
2021Q4
2022Q1


In [10]:
trade_attr[trade_attr.id == '5762439']

,id,applicant_type,credit_pull_id,sofi_num_trade_psl,sofi_num_trade_psl_open,sofi_num_trade_psl_closed,sofi_num_trade_list_psl,sofi_num_trade_list_psl_open,sofi_num_trade_list_psl_closed,sofi_num_trade_psl_1month,...,sofi_avg_term_psl_closed_6month,sofi_avg_term_list_psl_6month,sofi_avg_term_list_psl_open_6month,sofi_avg_term_list_psl_closed_6month,sofi_avg_term_psl_12month,sofi_avg_term_psl_open_12month,sofi_avg_term_psl_closed_12month,sofi_avg_term_list_psl_12month,sofi_avg_term_list_psl_open_12month,sofi_avg_term_list_psl_closed_12month
